In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import os
from functools import partial
from multiprocessing import Pool

from fastdtw import fastdtw, dtw
from sklearn import svm
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score

In [2]:
reduced = pd.read_csv("reduced.csv", index_col="date")
d = reduced.apply(np.log).diff()[1:]
data = []
for podatak in d["ABT"]:
    data.append(podatak)
d.shape

(969, 368)

In [3]:
L = len(data)
l = 20
s = 1

X = np.stack([data[i:i + l] for i in range(L // 2 - l - s)])
Y = np.stack([data[i + l + s - 1] > data[i + l - 1] for i in range(L // 2 - l - s)]).astype(np.float32)
##ako je sljedeći element veći od zadnjeg clana niza onda dobiva 1 inace 0

X_test = np.stack([data[i:i + l] for i in range(L // 2, L - l - s)])
Y_test = np.stack([data[i + l + s - 1] > data[i + l - 1] for i in range(L // 2, L - l - s)]).astype(np.float32)
print(len(X[0]))
print(X)
print(Y_test)

20
[[-0.00973507 -0.0136801   0.0093228  ... -0.0067644  -0.01366764
   0.00429124]
 [-0.0136801   0.0093228   0.02346702 ... -0.01366764  0.00429124
  -0.0048648 ]
 [ 0.0093228   0.02346702 -0.00053333 ...  0.00429124 -0.0048648
  -0.01765348]
 ...
 [-0.00954247  0.00521627  0.00367846 ... -0.01732639  0.00210349
  -0.00886456]
 [ 0.00521627  0.00367846 -0.00237864 ...  0.00210349 -0.00886456
  -0.0160277 ]
 [ 0.00367846 -0.00237864  0.00905179 ... -0.00886456 -0.0160277
   0.00579961]]
[0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0.
 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0.
 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0.
 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1.
 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0.
 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0.

In [4]:
len(X), len(X_test), Y.mean(), Y_test.mean()

(463, 464, 0.49892008, 0.46120688)

In [5]:
n = len(X)
m = len(X_test)
kernel_train = np.empty((n, n))
kernel_test = np.empty((m, n))

In [6]:
t0 = time.time()
for i in range(n):
    for j in range(i+1):
        if i == j:
            kernel_train[i][j] = 0.0
        else:
            kernel_train[i][j] = kernel_train[j][i] = fastdtw(X[i], X[j], radius = 5, dist = 2)[0]


for i in range(m):
    for j in range(n):           
        kernel_test[i][j] = fastdtw(X_test[i], X[j], radius = 5, dist = 2)[0]
                      
print(time.time()-t0)

5317.030606985092


In [8]:
kernel_train = np.load("kernel_train_univariant_ABT.npy")

In [10]:
kernel_test = np.load("kernel_test_univariant_ABT.npy")

In [11]:
c = 10
gamma = 25
kernel = np.exp(-kernel_train/gamma)
testkernel = np.exp(-kernel_test/gamma)
clf = svm.SVC(kernel='precomputed', C=c)
clf.fit(kernel, Y)
res = clf.predict(testkernel)
print('accuracy: ', np.mean(res == Y_test))

accuracy:  0.7241379310344828
